In [1]:
import pandas as pd
import json
from tqdm import tqdm
import math

from return_nearby_amenities import search_nearby_amenities, valid_amenities

import dask.dataframe as dd

In [2]:
listings = pd.read_csv('./data/processed/20231116/processed.csv')
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48729 entries, 0 to 48728
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                48729 non-null  int64  
 1   name              48729 non-null  object 
 2   url               48729 non-null  object 
 3   price             48729 non-null  object 
 4   num_bedroom       48304 non-null  float64
 5   num_bathroom      48001 non-null  float64
 6   cost_psf          47955 non-null  float64
 7   address           37597 non-null  object 
 8   road_name         37547 non-null  object 
 9   building          30414 non-null  object 
 10  postal_code       33697 non-null  float64
 11  latitude          37597 non-null  float64
 12  longitude         37597 non-null  float64
 13  floor_area        48729 non-null  int64  
 14  land_area         9166 non-null   float64
 15  walk_destination  40209 non-null  object 
 16  walk_distance_m   40209 non-null  float6

In [3]:
listing_dask = dd.from_pandas(listings, npartitions=10)

In [4]:
def get_amenities(row,amenity):
    if math.isnan(row['latitude']):
        return False
    
    result = search_nearby_amenities(
        lat1=row['latitude'],
        long1=row['longitude'],
        type=amenity,
        threshold=3
    ) 
    # print(len(result)>0)
    if len(result)>0:
        return True


In [5]:
results = listing_dask.apply(lambda x: get_amenities(x, 'college'), axis=1, meta=(None, bool)).compute()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

KeyboardInterrupt: 

In [5]:
for amenity in tqdm(valid_amenities):
    print(amenity)
    results = listing_dask.apply(lambda x: get_amenities(x, amenity), axis=1).compute()

  0%|          | 0/13 [00:00<?, ?it/s]

/Users/shaojieee/.pyenv/versions/3.11.5/envs/housing_proj/lib/python3.11/site-packages/dask/dataframe/core.py:6150: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))


college


  8%|▊         | 1/13 [00:03<00:36,  3.04s/it]/Users/shaojieee/.pyenv/versions/3.11.5/envs/housing_proj/lib/python3.11/site-packages/dask/dataframe/core.py:6150: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))


secondary_school


  8%|▊         | 1/13 [00:16<03:23, 16.96s/it]


KeyboardInterrupt: 

In [13]:
listings = listing_dask.compute()

KeyboardInterrupt: 

In [ ]:
listings

In [ ]:
amenities = {
    'property_id': [],
    'amenity_type': [],
    'amenity_name': [],
    'latitude': [],
    'longitude': [],
    'distance_km': [],
}

for amenity in tqdm(valid_amenities):
    listings[f'has_{amenity}'] = False
    for i, row in tqdm(listings.iterrows()):
        if math.isnan(row['latitude']):
            continue
        
        result = search_nearby_amenities(
            lat1=row['latitude'],
            long1=row['longitude'],
            type=amenity,
            threshold=3
        )

        listings.at[i, f'has_{amenity}'] = (len(result)>0)

        for result in result:
            amenities['property_id'].append(row['id'])
            amenities['amenity_type'].append(amenity)
            amenities['amenity_name'].append(result['name'])
            amenities['latitude'].append(result['lat'])
            amenities['longitude'].append(result['long'])
            amenities['distance_km'].append(result['distance_km'])

In [3]:
amenities = {
    'property_id': [],
    'amenity_type': [],
    'amenity_name': [],
    'latitude': [],
    'longitude': [],
    'distance_km': [],
}

for amenity in tqdm(valid_amenities):
    listings[f'has_{amenity}'] = False
    for i, row in tqdm(listings.iterrows()):
        if math.isnan(row['latitude']):
            continue
        
        result = search_nearby_amenities(
            lat1=row['latitude'],
            long1=row['longitude'],
            type=amenity,
            threshold=3
        )

        listings.at[i, f'has_{amenity}'] = (len(result)>0)

        for result in result:
            amenities['property_id'].append(row['id'])
            amenities['amenity_type'].append(amenity)
            amenities['amenity_name'].append(result['name'])
            amenities['latitude'].append(result['lat'])
            amenities['longitude'].append(result['long'])
            amenities['distance_km'].append(result['distance_km'])

48729it [06:22, 127.43it/s]0<?, ?it/s]
32780it [05:00, 108.91it/s]2<1:16:28, 382.41s/it]
  8%|▊         | 1/13 [11:23<2:16:40, 683.38s/it]


KeyboardInterrupt: 

In [31]:
listings.to_csv('./final.csv', index=False)